In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]

CHAN = ROOT / "chan_data"
SEG_FAST = CHAN / "segments" / "fast"
TREND_IN = CHAN / "trend_practical"
TREND_SIG_OUT = CHAN / "trend_signals"
MAP_SEG = CHAN / "mapped" / "segments_fast"
DATA = ROOT / "DATA"
BEICHI_OUT = CHAN / "beichi"

TREND_SIG_OUT.mkdir(parents=True, exist_ok=True)
BEICHI_OUT.mkdir(parents=True, exist_ok=True)

symbols = ["BTCUSDT","ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

from chan.signals.trend_signal import generate_trend_signals

def _read_trend(sym, tf):
    p = TREND_IN / sym / f"{tf}_trend.csv"
    if not p.exists():
        return None
    return pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])

def _read_segments(sym, tf):
    p = SEG_FAST / sym / f"{tf}_segments.csv"
    if not p.exists():
        return None
    return pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])

rows = []
for sym in symbols:
    for tf in tfs:
        tr = _read_trend(sym, tf)
        seg = _read_segments(sym, tf)
        if tr is None or seg is None or tr.empty or seg.empty:
            print(f"[skip] trend_signal {sym}-{tf}: missing/empty")
            continue
        sig = generate_trend_signals(tr, seg, use_midpoint=True, min_duration_days=1)
        out_dir = TREND_SIG_OUT / sym
        out_dir.mkdir(parents=True, exist_ok=True)
        p_out = out_dir / f"{tf}_trend_signal.csv"
        sig.to_csv(p_out, index=False)
        rows.append(dict(symbol=sym, tf=tf, n=len(sig), out=str(p_out)))
        print(f"[saved] trend_signal {sym}-{tf}: n={len(sig)}")

print("[done] trend signals saved.")
display(pd.DataFrame(rows))

[saved] trend_signal BTCUSDT-5m: n=7
[saved] trend_signal BTCUSDT-15m: n=7
[saved] trend_signal BTCUSDT-30m: n=11
[saved] trend_signal BTCUSDT-1h: n=11
[saved] trend_signal BTCUSDT-4h: n=4
[saved] trend_signal ETHUSDT-5m: n=6
[saved] trend_signal ETHUSDT-15m: n=2
[saved] trend_signal ETHUSDT-30m: n=4
[saved] trend_signal ETHUSDT-1h: n=12
[saved] trend_signal ETHUSDT-4h: n=2
[done] trend signals saved.


symbol   tf   n                                                out
0  BTCUSDT   5m   7  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
1  BTCUSDT  15m   7  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
2  BTCUSDT  30m  11  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
3  BTCUSDT   1h  11  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
4  BTCUSDT   4h   4  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
5  ETHUSDT   5m   6  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
6  ETHUSDT  15m   2  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
7  ETHUSDT  30m   4  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
8  ETHUSDT   1h  12  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...
9  ETHUSDT   4h   2  /Users/cc/PycharmProjects/ChanLab/chan_data/tr...

In [1]:
from pathlib import Path
import json
import pandas as pd

from chan.signals.beichi import (
    detect_beichi_over_segments,
    detect_beichi_over_ranges,
)

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]

DATA = ROOT / "DATA"
CHAN = ROOT / "chan_data"
SEG_FAST = CHAN / "segments" / "fast"
ZS_HIER = CHAN / "zhongshu_hierarchy_fast"
OUT = CHAN / "signals" / "beichi"
OUT.mkdir(parents=True, exist_ok=True)

symbols = ["BTCUSDT","ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

def _read_kline(sym, tf):
    p = DATA / sym / f"{tf}.csv"
    df = pd.read_csv(p)
    if "timestamp" not in df.columns:
        for c in ("time","datetime","ts"):
            if c in df.columns:
                df = df.rename(columns={c:"timestamp"})
                break
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
    need = {"timestamp","open","high","low","close"}
    miss = need - set(df.columns)
    if miss: raise ValueError(f"K线缺列: {miss} @ {p}")
    if "volume" not in df.columns:
        df["volume"] = 0.0
    return df

def _read_segments(sym, tf):
    p = SEG_FAST / sym / f"{tf}_segments.csv"
    df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    df["timestamp_start"] = pd.to_datetime(df["timestamp_start"], utc=True)
    df["timestamp_end"] = pd.to_datetime(df["timestamp_end"], utc=True)
    return df

def _read_zs_l1(sym, tf):
    p = ZS_HIER / sym / f"{tf}_L1.csv"
    if not p.exists():
        return pd.DataFrame()
    df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    return df

def _events_to_df(evts):
    if not evts:
        return pd.DataFrame(columns=["ts","etype","price","score","payload"])
    rows = []
    for e in evts:
        rows.append(dict(
            ts = pd.to_datetime(e["ts"], utc=True),
            etype = e.get("etype",""),
            price = float(e.get("price", float("nan"))),
            score = float(e.get("payload",{}).get("score", float("nan"))),
            payload = json.dumps(e.get("payload",{}), ensure_ascii=False)
        ))
    return pd.DataFrame(rows).sort_values("ts").reset_index(drop=True)

logs = []
for sym in symbols:
    for tf in tfs:
        try:
            k = _read_kline(sym, tf)
        except Exception as e:
            print(f"[skip] {sym}-{tf} kline err: {e}")
            continue

        try:
            seg = _read_segments(sym, tf)
        except Exception as e:
            print(f"[skip] beichi-trend {sym}-{tf} segments err: {e}")
            seg = pd.DataFrame()

        trend_evts = []
        if not seg.empty:
            trend_evts = detect_beichi_over_segments(
                kline=k, segments=seg, enable_macd=True
            )
        df_trend = _events_to_df(trend_evts)
        out_dir = OUT / sym
        out_dir.mkdir(parents=True, exist_ok=True)
        p_trend = out_dir / f"{tf}_beichi_trend.csv"
        df_trend.to_csv(p_trend, index=False)
        print(f"[saved] beichi-trend {sym}-{tf}: n={len(df_trend)}")

        zs = _read_zs_l1(sym, tf)
        range_evts = []
        if not zs.empty:
            range_evts = detect_beichi_over_ranges(
                kline=k, zs_l1=zs, enable_macd=True, band_ratio=0.002
            )
        df_range = _events_to_df(range_evts)
        p_range = out_dir / f"{tf}_beichi_range.csv"
        df_range.to_csv(p_range, index=False)
        print(f"[saved] beichi-range {sym}-{tf}: n={len(df_range)}")

        df_all = pd.concat([df_trend, df_range], ignore_index=True).sort_values("ts").reset_index(drop=True)
        p_all = out_dir / f"{tf}_beichi_all.csv"
        df_all.to_csv(p_all, index=False)

        logs.append(dict(symbol=sym, tf=tf, n_trend=len(df_trend), n_range=len(df_range), out=str(p_all)))

print("[done] beichi signals saved.")
pd.DataFrame(logs)

[saved] beichi-trend BTCUSDT-5m: n=40
[saved] beichi-range BTCUSDT-5m: n=23
[saved] beichi-trend BTCUSDT-15m: n=44
[saved] beichi-range BTCUSDT-15m: n=20
[saved] beichi-trend BTCUSDT-30m: n=39
[saved] beichi-range BTCUSDT-30m: n=20
[saved] beichi-trend BTCUSDT-1h: n=21
[saved] beichi-range BTCUSDT-1h: n=17
[saved] beichi-trend BTCUSDT-4h: n=1
[saved] beichi-range BTCUSDT-4h: n=7
[saved] beichi-trend ETHUSDT-5m: n=151
[saved] beichi-range ETHUSDT-5m: n=83
[saved] beichi-trend ETHUSDT-15m: n=88
[saved] beichi-range ETHUSDT-15m: n=47
[saved] beichi-trend ETHUSDT-30m: n=57
[saved] beichi-range ETHUSDT-30m: n=29
[saved] beichi-trend ETHUSDT-1h: n=21
[saved] beichi-range ETHUSDT-1h: n=10
[saved] beichi-trend ETHUSDT-4h: n=6
[saved] beichi-range ETHUSDT-4h: n=5
[done] beichi signals saved.


symbol   tf  n_trend  n_range  \
0  BTCUSDT   5m       40       23   
1  BTCUSDT  15m       44       20   
2  BTCUSDT  30m       39       20   
3  BTCUSDT   1h       21       17   
4  BTCUSDT   4h        1        7   
5  ETHUSDT   5m      151       83   
6  ETHUSDT  15m       88       47   
7  ETHUSDT  30m       57       29   
8  ETHUSDT   1h       21       10   
9  ETHUSDT   4h        6        5   

                                                 out  
0  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
1  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
2  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
3  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
4  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
5  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
6  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
7  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
8  /Users/cc/PycharmProjects/ChanLab/chan_data/si...  
9  /Users/cc/PycharmProjects/ChanLab/chan_data/si...

In [4]:

from pathlib import Path
import pandas as pd

def _get_root():
    R = Path.cwd().resolve()
    return R.parents[0] if R.name == "Note" else R

ROOT = _get_root()
CHAN = ROOT / "chan_data"
BEICHI_DIR = CHAN / "signals" / "beichi"

def qc_beichi(sym: str, tf: str):
    p = BEICHI_DIR / sym / f"{tf}_beichi_all.csv"
    if not p.exists():
        raise FileNotFoundError(f"未找到文件: {p}\n"
                                f"请确认 09B 成功落盘，或当前工作目录是否为工程根/Note。")
    df = pd.read_csv(p, parse_dates=["ts"])

    assert df["ts"].dt.tz is not None, "ts 必须是 tz-aware（UTC）"
    assert df["ts"].is_monotonic_increasing, "ts 应该从小到大"
    assert df["etype"].str.startswith("SIG.BEICHI").all(), "etype 前缀应为 SIG.BEICHI"
    print(f"[ok] {sym}-{tf}: n={len(df)}")
    print(df["score"].describe().round(3))
    return df

for s in ["BTCUSDT","ETHUSDT"]:
    for tf in ["5m","15m","30m","1h","4h"]:
        qc_beichi(s, tf)

[ok] BTCUSDT-5m: n=63
count    63.000
mean      2.241
std       0.906
min       0.889
25%       1.418
50%       2.484
75%       2.987
max       4.000
Name: score, dtype: float64
[ok] BTCUSDT-15m: n=64
count    64.000
mean      2.273
std       1.004
min       0.824
25%       1.344
50%       2.379
75%       3.341
max       4.000
Name: score, dtype: float64
[ok] BTCUSDT-30m: n=59
count    59.000
mean      2.399
std       0.938
min       0.964
25%       1.361
50%       2.748
75%       3.248
max       3.784
Name: score, dtype: float64
[ok] BTCUSDT-1h: n=38
count    38.000
mean      2.483
std       0.819
min       0.869
25%       1.614
50%       2.651
75%       3.159
max       3.800
Name: score, dtype: float64
[ok] BTCUSDT-4h: n=8
count    8.000
mean     2.202
std      1.094
min      1.022
25%      1.281
50%      1.833
75%      3.361
max      3.642
Name: score, dtype: float64
[ok] ETHUSDT-5m: n=234
count    234.000
mean       2.441
std        0.889
min        0.778
25%        1.582
50%      

In [6]:
import ast, json
from pathlib import Path
import pandas as pd

def _get_root():
    R = Path.cwd().resolve()
    return R.parents[0] if R.name == "Note" else R

ROOT = _get_root()
CHAN = ROOT / "chan_data"
BEICHI_DIR = CHAN / "signals" / "beichi"
SEG_DIR = CHAN / "segments" / "fast"

def _parse_payload(x):
    if isinstance(x, dict):
        return x
    try:
        return json.loads(x)
    except Exception:
        try:
            return ast.literal_eval(x)
        except Exception:
            return {}

def peek_with_segments(sym: str, tf: str, n: int = 5, only_trend: bool = True):
    p_evt = BEICHI_DIR / sym / f"{tf}_beichi_trend.csv"
    p_seg = SEG_DIR / sym / f"{tf}_segments.csv"
    if not p_evt.exists():
        raise FileNotFoundError(f"事件文件不存在: {p_evt}")
    if not p_seg.exists():
        raise FileNotFoundError(f"线段文件不存在: {p_seg}")

    e = pd.read_csv(p_evt, parse_dates=["ts"])
    seg = pd.read_csv(p_seg, parse_dates=["timestamp_start","timestamp_end"])

    e["ts"] = pd.to_datetime(e["ts"], utc=True, errors="coerce")
    seg["timestamp_start"] = pd.to_datetime(seg["timestamp_start"], utc=True, errors="coerce")
    seg["timestamp_end"]   = pd.to_datetime(seg["timestamp_end"],   utc=True, errors="coerce")

    if only_trend:
        e = e[e["etype"].str.startswith("SIG.BEICHI.TREND")]

    pay = e["payload"].apply(_parse_payload)
    e["seg_idx"] = pay.apply(lambda d: d.get("seg_idx"))
    e = e.dropna(subset=["seg_idx"]).copy()

    def _abc(v):
        if isinstance(v, (list, tuple)) and len(v) >= 3:
            return int(v[0]), int(v[1]), int(v[2])
        return None

    e["abc"] = e["seg_idx"].apply(_abc)
    e = e.dropna(subset=["abc"]).copy()

    e = e.sort_values("ts").tail(n)

    for _, r in e.iterrows():
        a, b, c = r["abc"]
        if c >= len(seg):
            print(f"[warn] seg 索引越界: seg_len={len(seg)}, got c={c} @ {r['ts']}")
            continue
        ce = pd.to_datetime(seg.iloc[c]["timestamp_end"], utc=True)
        print(
            f"{sym}-{tf}  {r['ts']}  {r['etype']}  "
            f"score={float(r['score']):.2f}  seg=({a},{b},{c})  C_end={ce}"
        )

peek_with_segments("BTCUSDT", "15m", n=5)

BTCUSDT-15m  2025-09-18 03:30:00+00:00  SIG.BEICHI.TREND.UP  score=1.51  seg=(103,104,105)  C_end=2025-09-18 03:30:00+00:00
BTCUSDT-15m  2025-09-24 04:00:00+00:00  SIG.BEICHI.TREND.DOWN  score=3.40  seg=(104,105,106)  C_end=2025-09-24 04:00:00+00:00
BTCUSDT-15m  2025-10-03 16:30:00+00:00  SIG.BEICHI.TREND.UP  score=2.60  seg=(108,109,110)  C_end=2025-10-03 16:30:00+00:00
BTCUSDT-15m  2025-10-05 05:30:00+00:00  SIG.BEICHI.TREND.UP  score=2.83  seg=(109,110,111)  C_end=2025-10-05 05:30:00+00:00
BTCUSDT-15m  2025-10-09 17:00:00+00:00  SIG.BEICHI.TREND.DOWN  score=1.52  seg=(112,113,114)  C_end=2025-10-09 17:00:00+00:00


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]

CHAN = ROOT / "chan_data"
TREND_DIR = CHAN / "trend_signals"
BEICHI_DIR = CHAN / "signals" / "beichi"
MERGED_DIR = CHAN / "signals" / "merged"

def merge_trend_and_beichi(sym: str, tf: str, *, dedup: bool = False) -> pd.DataFrame:
    p_tr = TREND_DIR / sym / f"{tf}_trend_signal.csv"
    p_bc = BEICHI_DIR / sym / f"{tf}_beichi_all.csv"

    if not p_tr.exists():
        raise FileNotFoundError(f"trend file missing: {p_tr}")
    if not p_bc.exists():
        raise FileNotFoundError(f"beichi file missing: {p_bc}")

    tr = pd.read_csv(p_tr, parse_dates=["ts"])
    bc = pd.read_csv(p_bc, parse_dates=["ts"])

    tr["src"] = "trend"
    tr = tr.rename(columns={"signal":"label"}).assign(etype="SIG.TREND")

    bc["src"] = "beichi"
    if "price" in bc.columns:
        bc = bc.rename(columns={"price":"px"})
    bc = bc.assign(label="beichi")

    left  = tr[["ts","etype","label","direction","src"]].assign(px=np.nan, score=np.nan)
    right = bc[["ts","etype","label","px","score","src"]].assign(direction=np.nan)
    out = pd.concat([left, right], ignore_index=True)

    if dedup:
        out = (out.sort_values(["ts","src"])
                 .drop_duplicates(subset=["ts","etype","label"], keep="first"))

    out = out.sort_values("ts").reset_index(drop=True)

    p_out_dir = MERGED_DIR / sym
    p_out_dir.mkdir(parents=True, exist_ok=True)
    out_path = p_out_dir / f"{tf}_merged_signals.csv"
    out.to_csv(out_path, index=False)

    n_tr = int((out["src"]=="trend").sum())
    n_bc = int((out["src"]=="beichi").sum())
    print(f"[merged] {sym}-{tf}: trend={n_tr}, beichi={n_bc}, total={len(out)} -> {out_path}")

    return out

def merged_summary(sym: str, tf: str):
    p = MERGED_DIR / sym / f"{tf}_merged_signals.csv"
    df = pd.read_csv(p, parse_dates=["ts"])
    print(df["src"].value_counts())
    print(df.groupby("etype")["src"].count().sort_values(ascending=False).head())
    print(df.tail(5)[["ts","etype","label","src","direction","px","score"]])

symbols = ["BTCUSDT","ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

for s in symbols:
    for tf in tfs:
        try:
            merge_trend_and_beichi(s, tf, dedup=False)
        except Exception as e:
            print(f"[skip] {s}-{tf}: {e}")


p_check = MERGED_DIR / "BTCUSDT" / "15m_merged_signals.csv"
if p_check.exists():
    merged_summary("BTCUSDT","15m")
else:
    print(f"[skip] summary: not found -> {p_check}")


[skip] BTCUSDT-5m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/BTCUSDT/5m_trend_signal.csv
[skip] BTCUSDT-15m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/BTCUSDT/15m_trend_signal.csv
[skip] BTCUSDT-30m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/BTCUSDT/30m_trend_signal.csv
[skip] BTCUSDT-1h: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/BTCUSDT/1h_trend_signal.csv
[skip] BTCUSDT-4h: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/BTCUSDT/4h_trend_signal.csv
[skip] ETHUSDT-5m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/ETHUSDT/5m_trend_signal.csv
[skip] ETHUSDT-15m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/ETHUSDT/15m_trend_signal.csv
[skip] ETHUSDT-30m: trend file missing: /Users/cc/PycharmProjects/ChanLab/chan_data/trend_signals/ETHUSDT/30m_trend_sig